In [1]:
import pandas as pd

In [2]:
users = pd.read_csv("data/pur.csv")
items = pd.read_csv("KNN_dataset.csv")
target = pd.read_csv("data/target_group.csv")
visited = pd.read_csv("data/visited_products.csv")

In [3]:
users.head()

,customer_id,product_id,timestamp,price,count
0,0d1b7397-7d3c-44c0-9efc-d38bf197828b,954f9f2c-d3ca-4236-ac9a-4ea7bcf09305,2021-03-22 04:45:17.308966+00:00,314.26,1.0
1,81ccae7e-e496-4997-a289-4669bf53f33e,75b281e5-8a16-42cb-9ae0-9a98db7a2c40,2021-03-22 04:59:26.404186+00:00,612.12,2.0
2,50777c55-8dd6-4309-a5ca-26e66c8a8279,0112dec8-47f5-4c2c-9109-571e2dbb6345,2021-03-22 05:13:58.989935+00:00,238.70,2.0
3,50777c55-8dd6-4309-a5ca-26e66c8a8279,0fa25a2d-2aa1-4397-82f1-5a64f3b1272d,2021-03-22 05:13:58.991935+00:00,314.26,1.0
4,647d269f-b18f-4558-8653-93369d862ec9,81c01216-55a9-4588-a722-bccf0bf35fd5,2021-03-22 05:32:16.083698+00:00,478.92,4.0


In [4]:
from sklearn.neighbors import KDTree

In [5]:
%%time
columns = items.columns[2:]
X = items[columns]
tree = KDTree(X)


CPU times: user 76.2 ms, sys: 6.51 ms, total: 82.7 ms
Wall time: 82 ms


In [ ]:
result = {
"customer_id" : list(),
"product_id" : list()
}
for index,row in target.iterrows():
    if (index + 1) % 100 == 0:
        print(index)
    v = visited[visited['customer_id'] == row['customer_id']]
    b = users[users['customer_id'] == row['customer_id']]
    find = dict()
    i = items[items['product_id'].isin(b['product_id'])]
    if len(i) > 0:
        i = i[columns].fillna(0).mean().values
        dist, ind = tree.query([i], k=5)

        for j in ind[0][1:]:
            f_id = items.loc[j]['product_id']
            if f_id in find:
                find[f_id] += 1
            else:
                find[f_id] = 1

    i = items[items['product_id'].isin(v['product_id'])]
    if len(i) > 0:
        i = i[columns].fillna(0).mean().values
        dist, ind = tree.query([i], k=15)
        for j in ind[0][1:]:
            f_id = items.loc[j]['product_id']
            if f_id in find:
                find[f_id] += 1
            else:
                find[f_id] = 0.8

    find = dict(sorted(find.items(), key=lambda x: x[1]))
    for i in range(5):
        if len(find.keys()) > i:
            result['customer_id'].append(row['customer_id'])
            result['product_id'].append(list(find.keys())[i])
        else:
            result['customer_id'].append(row['customer_id'])
            result['product_id'].append(items.sample(1)['product_id'])


99
199
299
399
499
599
699
799
899
999
1099
1199
1299
1399
1499
1599
1699
1799
1899
1999
2099
2199
2299
2399
2499
2599
2699
2799
2899
2999
3099
3199
3299
3399
3499
3599
3699
3799
3899
3999
4099
4199
4299
4399
4499
4599
4699
4799
4899
4999
5099
5199
5299
5399
5499
5599
5699
5799
5899
5999
6099
6199
6299
6399
6499
6599
6699
6799
6899
6999
7099
7199
7299
7399
7499
7599
7699
7799


In [ ]:
# 22:42
results = pd.DataFrame(data=result)
restuls.to_csv("pls_funguj.csv")